In [2]:
!pwd

/home/soyoung/New_Freelancer/freelancer


In [6]:
# !cp ../freelancer/ /home/soyoung/New_Freelancer

In [4]:
# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

import pandas as pd
import os
from time import time, ctime

import concurrent.futures

from lxml import etree
from bs4 import BeautifulSoup
import json
import re

In [7]:
# df=pd.read_csv('data/escrow_ID_name.csv')
# df.head()

In [9]:
folder='bidder_pages/'
cont = os.listdir(folder)
len(cont)

1882

In [10]:
with open(folder+"superwanchiu.html",'r') as fl:
        text=fl.read()
print len(text)
test=etree.HTML(text)
test

183061


<Element html at 0x7f9c945fe758>

In [11]:
soup = BeautifulSoup(text, "lxml")
type(soup)

bs4.BeautifulSoup

### use etree instead of beautifulsoup
### performance comparison

https://en.wikipedia.org/wiki/Orders_of_magnitude_(time)

In [12]:
%%time
username = test.xpath('//div[@class="profile-user-name"]/text()')[0].split()[0]
print username

@superwanchiu
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 844 µs


In [13]:
%%time
username = soup.findAll('div', {'class': 'profile-user-name'})[0].text
username=os.linesep.join([s for s in username.strip().splitlines(True) if s.strip("\r\n").strip()])
print username

@superwanchiu
CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 23.2 ms


In [ ]:
//*[@id="sticky-start"]/div/section[2]/div[1]/h1

In [17]:
%%time
name = test.xpath('//h1[@class="profile-intro-username"]/text()')[0].strip()
print name

Lizu Lu
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 625 µs


#### can be up to 50 times faster

In [77]:
30.7*1000/623

49.2776886035313

In [212]:
location = soup.findAll('span', {'class': 'profile-location-name'})
location = location[0].text
location = os.linesep.join([s for s in location.strip().splitlines(True) if s.strip("\r\n").strip()])
print location

India


In [213]:
location = test.xpath('//span[@class="profile-location-name"]/text()')[0].split()[0]
print location

India


In [214]:
membership = soup.findAll('div', {'class': 'profile-membership-length'})
membership = membership[0].text
membership = os.linesep.join([s for s in membership.strip().splitlines(True) if s.strip("\r\n").strip()])
##change
membership

u'Member since May, 2005'

In [215]:
membership=test.xpath('//div[@class="profile-membership-length"]/text()')[0].strip()
membership

'Member since May, 2005'

#### some of the code doesn't work, may be the source page structure changed

In [223]:
timeOffset = soup.findAll('span', {'class': 'timeOffset'})
timeOffset = timeOffset[0].text
timeOffset = os.linesep.join([s for s in timeOffset.strip().splitlines(True) if s.strip("\r\n").strip()])
timeOffset

IndexError: list index out of range

In [218]:
timeOffset=test.xpath('//span[@class="timeOffset"]/text()')
timeOffset

[]

In [224]:
city=test.xpath('//span[@class="PageProfile-info-locality"]/text()')[0].strip()
city

'Bhopal'

In [227]:
search_languages = soup.findAll('span', {'class': 'search_languages'})
search_languages 
# search_languages = os.linesep.join([s for s in search_languages.strip().splitlines(True) if s.strip("\r\n").strip()])

[]

# I find most of the data are in the last javascript tag < script> stored as json format

### please open a html and scroll down to the end to find the last < script> tag and check it out

### so I wrote a regular expression script to get the json data

### the json data contains some other important info besides the info shows in the user page

### I browse around, the htmls I opened all have that json data in the end, so I think I'll get all the useful info from the json data first of all the user profile htmls

In [155]:
%%time
script = test.xpath('//script')[-1]


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 225 µs


In [320]:
data = re.search(r"__export\('profileServerData',(.*?)\n", script.text)
# print data.group(0)
# print data.group(1)[:-2]
jsondata=data.group(1)[:-2]
print type(jsondata), len(jsondata)
json_dict = json.loads(jsondata)
# json_dict

<type 'str'> 20071


### the profile has up to 240 fields in the json data, althought most of the htmls' most field are null, yet some fields are useful, such as profile's rating


In [315]:
json_dict['about']['reputation']

{u'earnings_score': 9.284,
 u'entire_history': {u'all': 472,
  u'category_ratings': {u'communication': 4.719101777198227,
   u'expertise': 4.7995511082961135,
   u'hire_again': 4.765460744223163,
   u'professionalism': 4.8186139532620285,
   u'quality': 4.74718687934979},
  u'complete': 326,
  u'completion_rate': 0.690677966101695,
  u'earnings': None,
  u'incomplete': 146,
  u'incomplete_reviews': 18,
  u'on_budget': 0.9597868189256311,
  u'on_time': 0.7831042238740723,
  u'overall': 4.7699828924658645,
  u'positive': 0.32386695583965214,
  u'rehire_rate': 0.14469453376205788,
  u'reviews': 250},
 u'job_history': {u'count_other': 273,
  u'job_counts': [{u'count': 126,
    u'job': {u'active_project_count': None,
     u'category': {u'id': 1, u'name': u'Websites, IT & Software'},
     u'id': 3,
     u'local': False,
     u'name': u'PHP',
     u'seo_info': None,
     u'seo_url': None}},
   {u'count': 112,
    u'job': {u'active_project_count': None,
     u'category': {u'id': 3, u'name': u'

In [234]:
len(json_dict)

15

#### func to parse json to csv 

In [235]:
def flatten_dict(data, layers=1, drop_deeper=True):
        """
        takes in a dictionary and will flatten it with level_1.level_2 as the key, for however many levels are
        specified. If specified (true by default), anything deeper than the specified level will be dropped from the
        dictionary outright.

        :param data:
        :param layers:
        :param drop_deeper:
        :return:
        """
        for _ in range(layers):
            data = [(k, v) if not isinstance(v, dict) else [(k + '.' + k2, v2) for k2, v2 in v.items()] for k, v in
                    data.items()]
            data = [item for sublist in data for item in sublist if isinstance(sublist, list)] + [y for y in data if
                                                                                                  not isinstance(y,
                                                                                                                 list)]
            data = dict(data)

        if drop_deeper:
            data = {k: v for k, v in data.items() if not isinstance(v, dict) or isinstance(v, list)}

        return data

In [251]:
# len(flatten_dict(json_dict, layers=5))

In [316]:
# flatten_dict(json_dict, layers=1)

In [318]:
# l1=flatten_dict(json_dict, layers=5)
# len(l1)

In [321]:
# print l1.keys()

In [306]:
l1['about.reputation.earnings_score']

9.284

In [308]:
l1['about.reputation.entire_history.all']

472

In [256]:
l1['about.profile_description']

u'Mobile & Web Application Development Company\n\nDelivering top-grade quality mobile app development solutions is something we at OM Software have been excelling in since the early launch of our business. We are well aware of the existing mobile app market trends and also keep a tab on the evolving trend of the industry, which is why when it comes to developing high-grade quality mobile apps, we consistently lead at the top position. As a result of our continued success in the business, we are being looked upon as one of the forerunners in the industry that offer comprehensive services and solutions for mobile apps.\n\nBasically, we are armed with seasoned web application development technicians who excel in developing A-one websites and web apps that perform best on all platforms.\n\nBeing a NASSCOM & ISO certified Company, we invariably strive our best to cater to your software needs from website development to mobile app development.'

In [257]:
l1['resume.certifications']

[{u'award_date': u'2011',
  u'certificate': u'North India\'s leading service organizations and professionals in the category of "Best Web Design & Development Services"',
  u'descr': u'\n\n',
  u'id': u'56439',
  u'organisation': u'',
  u'organization': u''},
 {u'award_date': u'2014',
  u'certificate': u'ISO Certification',
  u'descr': u'ISO 9001:2008 Certified Web Development Company',
  u'id': u'459024',
  u'organisation': u'',
  u'organization': u'ISO 9001:2008'},
 {u'award_date': u'2014',
  u'certificate': u'CRISIL SE 3B',
  u'descr': u'Certified by Crisil Rating Policy',
  u'id': u'459025',
  u'organisation': u'',
  u'organization': u'Crisil Rating Company'},
 {u'award_date': u'2014',
  u'certificate': u'Enterprenuer of the Month May 2014',
  u'descr': u'',
  u'id': u'459026',
  u'organisation': u'',
  u'organization': u'Silicon India'},
 {u'award_date': u'2014',
  u'certificate': u'NASSCOM Feature Member',
  u'descr': u'',
  u'id': u'459027',
  u'organisation': u'',
  u'organizat

In [250]:
l1['skills']

[{u'certified': False,
  u'id': 3,
  u'islocal': u'0',
  u'name': u'PHP',
  u'seo_url': u'php',
  u'usages': 174},
 {u'certified': False,
  u'id': 17,
  u'islocal': u'0',
  u'name': u'Website Design',
  u'seo_url': u'website_design',
  u'usages': 164},
 {u'certified': False,
  u'id': 335,
  u'islocal': u'0',
  u'name': u'HTML',
  u'seo_url': u'html',
  u'usages': 102},
 {u'certified': False,
  u'id': 20,
  u'islocal': u'0',
  u'name': u'Graphic Design',
  u'seo_url': u'graphic_design',
  u'usages': 91},
 {u'certified': False,
  u'id': 305,
  u'islocal': u'0',
  u'name': u'MySQL',
  u'seo_url': u'mysql',
  u'usages': 52},
 {u'certified': False,
  u'id': 38,
  u'islocal': u'0',
  u'name': u'SEO',
  u'seo_url': u'seo',
  u'usages': 51},
 {u'certified': False,
  u'id': 15,
  u'islocal': u'0',
  u'name': u'.NET',
  u'seo_url': u'dot_net',
  u'usages': 26},
 {u'certified': False,
  u'id': 5,
  u'islocal': u'0',
  u'name': u'ASP',
  u'seo_url': u'asp',
  u'usages': 24},
 {u'certified': False,

## full json data

In [322]:
json_dict

{u'abTests': [],
 u'about': {u'account_balances': None,
  u'address': None,
  u'avatar': u'/ppic/104226104/logo/70891/profile_logo_70891.jpg',
  u'avatar_cdn': u'//cdn5.f-cdn.com/ppic/104226104/logo/70891/profile_logo_70891.jpg',
  u'avatar_large': u'/ppic/104226102/logo/70891/SoIM2/profile_logo_.jpg',
  u'avatar_large_cdn': u'//cdn6.f-cdn.com/ppic/104226102/logo/70891/SoIM2/profile_logo_.jpg',
  u'badges': None,
  u'chosen_role': 1,
  u'closed': False,
  u'company': u'OMSOFTWARE  INTERNET SOLUTIONS PVT LTD',
  u'corporate': {u'founder_id': 24324137, u'status': 1},
  u'corporate_users': None,
  u'cover_image': {u'current_image': {u'description': None,
    u'height': 550,
    u'id': 76082,
    u'url': u'//cdn2.f-cdn.com/files/download/49379989/bc76c3.jpg',
    u'width': 1920},
   u'past_images': None},
  u'display_name': u'OMSOFTWARE  PVT LTD',
  u'email': None,
  u'employer_reputation': None,
  u'endorsements': None,
  u'first_name': None,
  u'force_verify': None,
  u'hourly_rate': 20,